# ggsdtpy

In [10]:
import math
import numpy as np
import pandas as pd
from scipy.stats import gennorm
from scipy.optimize import minimize, SR1

## likelihood function

In [11]:
def fit_ggsdt_logL(x, parameters):
    m2 =   x[0]
    alp2 = x[1]
    bet =  x[2]
    cri =  x[3:]
    
    nR_S1, nR_S2, n_ratings = parameters

    far = gennorm.cdf(cri, scale = 1, beta = bet, loc = 0)
    hr =  gennorm.cdf(cri - m2, scale = alp2, beta = bet, loc = 0)

    s1_exp = np.sum(nR_S1) * np.diff(far)
    s2_exp = np.sum(nR_S2) * np.diff(hr)
    
    s1_exp_number = np.hstack([np.sum(nR_S1) * far[0], s1_exp, np.sum(nR_S1) \
    - np.sum(nR_S1) * far[0] - np.sum(s1_exp)])
    s2_exp_number = np.hstack([np.sum(nR_S2) * hr[0],  s2_exp, np.sum(nR_S2) \
    - np.sum(nR_S2) * hr[0]  - np.sum(s2_exp)])

    logL = np.sum(nR_S2 * np.log(s2_exp_number / np.sum(nR_S2)) \
    + nR_S1 * np.log(s1_exp_number / np.sum(nR_S1)))
   
    if np.isinf(logL) or np.isnan(logL):
        logL = -1e+300 # returning "-inf" may cause optimize.minimize() to fail
    
    return -logL

## model fitting

In [12]:
def fit_ggsdt(nR_S1, nR_S2, add_constant = True):
    
    if (add_constant):
        nR_S1 = nR_S1 + (1 / len(nR_S1))
        nR_S2 = nR_S2 + (1 / len(nR_S2))
    
    n_ratings = int(len(nR_S1) / 2)
    ratingFAR = 1 - np.cumsum(nR_S1) / np.sum(nR_S1)
    ratingHR =  1 - np.cumsum(nR_S2) / np.sum(nR_S2)

    # set up initial guess for parameter values
    alp2 = 1
    bet = 2
    mu2 = gennorm.ppf(ratingHR[n_ratings - 1], scale = alp2, beta = bet) \
          - gennorm.ppf(ratingFAR[n_ratings - 1], scale = alp2, beta = bet)
    cri = -1 * gennorm.ppf(ratingFAR, scale = alp2, beta = bet)
    cri = cri[0:(2 * n_ratings - 1)]
    
    guess = np.hstack([mu2, alp2, bet, cri])

    # model fitting
    parameters = [nR_S1, nR_S2, n_ratings]
    
    fit = minimize(fit_ggsdt_logL, guess, args = parameters, method='trust-constr',
                       jac='2-point', hess=SR1())
                       
    m2 =   fit.x[0]
    alp2 = fit.x[1]
    bet =  fit.x[2]
    cri =  fit.x[3:]
    logL = -fit.fun
    sd1 =  math.sqrt((1**2 * math.gamma(3 / bet)) / math.gamma(1 / bet))
    sd2 =  math.sqrt((alp2**2 * math.gamma(3 / bet)) / math.gamma(1 / bet))
    kurt = (math.gamma(5 / bet) * math.gamma(1 / bet)) / math.gamma(3 / bet)**2 - 3

    est = pd.DataFrame()
    est["mu2"] = [m2]
    est["alpha2"] = [alp2]
    est["beta"] = [bet]
    est["loglike"] = [logL]
    est["sigma1"] = [sd1]
    est["sigma2"] = [sd2]
    est["kurtosis"] = [kurt]
    
    for i in range(2 * n_ratings - 1):
        est["c" + str(i + 1)] = [fit.x[3 + i]]
    
    return est

## examples

In [13]:
nR_S1 = np.array([170, 120, 50, 25, 45, 40])
nR_S2 = np.array([40, 50, 30, 20, 70, 240])

fit_ggsdt(nR_S1, nR_S2, add_constant = False)

<ipython-input-11-983aff62f560>:20: RuntimeWarning: invalid value encountered in log
  logL = np.sum(nR_S2 * np.log(s2_exp_number / np.sum(nR_S2)) \
<ipython-input-11-983aff62f560>:21: RuntimeWarning: invalid value encountered in log
  + nR_S1 * np.log(s1_exp_number / np.sum(nR_S1)))


,mu2,alpha2,beta,loglike,sigma1,sigma2,kurtosis,c1,c2,c3,c4,c5
0,1.090558,1.306446,1.781324,-1338.253235,0.75595,0.987609,0.257916,-0.224036,0.271486,0.500422,0.635745,1.012405
